In [ ]:
pip install requests beautifulsoup4

In [ ]:
pip install selenium webdriver-manager requests beautifulsoup4 lxml

In [ ]:
pip install lxml

In [ ]:
pip install tqdm

In [ ]:
import os
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = True  
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

url = 'https://vsco.co/feed'
driver.get(url)

scroll_pause_time = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
soup

In [ ]:
# links = soup.find_all("a", href=True)

# for link in links:
#     print(link['href'])

In [ ]:
base_url = "https://vsco.co"
links = soup.find_all("a", href=True)
for link in links:
    if '/media/' in link['href']:
        full_url = base_url +'/'+ link['href']
        print(full_url)


In [ ]:
from tqdm import tqdm
from PIL import Image

base_url = "https://vsco.co"
links = soup.find_all("a", href=True)
for link in tqdm(links):
    if '/media/' in link['href']:
        full_url = base_url +'/'+ link['href']
        print(full_url)
        url = full_url
        driver.get(url)

        scroll_pause_time = 2
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        linksa = soup.find_all("img", src=lambda src: src and 'jpg' in src)
        for img_tag in linksa:
            src_url = img_tag['src']
            src_url = "https:"+ src_url
            response = requests.get(src_url)
            if response.status_code == 200:
                file_name = src_url.split("/")[-1]
                with open(file_name, "wb") as f:
                    f.write(response.content)
                print("Image downloaded successfully as:", file_name)
                imm = Image.open(file_name)
                display(imm)
            else:
                print("Failed to download the image. Status code:", response.status_code)

